In [4]:
import cv2
import numpy as np
from loaddata import loaddata
from mediapipe_analysis import predict_nn as method_mediapipe
from sklearn import svm
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow.keras import losses, Sequential, layers

In [5]:
x_train, x_test, y_train, y_test = loaddata('1', '2', '3', train_size=0.8)

---
### HOG

In [6]:
winSize = (160, 160) 
blockSize = (80, 80)
blockStride = (40, 40)
cellSize = (80, 80)
Bin = 9 
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, Bin)
HOG = lambda img: hog.compute(img).flatten()

train_HOG = np.array([HOG(img) for img in x_train])

SVM = svm.SVC(kernel = 'linear')
SVM.fit(train_HOG, y_train)
del train_HOG

In [8]:
test_HOG = np.array([HOG(img) for img in x_test])
y_predict = SVM.predict(test_HOG)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.89      0.94      0.92       140
           1       0.97      0.95      0.96        59
           2       0.91      0.86      0.89        37
           3       0.97      0.94      0.96        34
           4       0.95      0.85      0.90        41

    accuracy                           0.92       311
   macro avg       0.94      0.91      0.92       311
weighted avg       0.92      0.92      0.92       311



In [9]:
method_SVM = lambda img: SVM.predict([HOG(img)])[0]

---
### CNN

In [5]:
def convModel():
    model = Sequential([
        layers.Conv2D(64, (3, 3), activation = 'relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation = 'relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation = 'relu'),
        layers.Flatten(),
        layers.Dense(10000, activation = 'relu'),
        layers.Dropout(0.2),
        layers.Dense(5, activation = 'softmax', name = "final")
    ])
    model.compile(optimizer = 'adam', loss = losses.SparseCategoricalCrossentropy(), metrics = ['accuracy'])
    return model

CNN = convModel()

In [7]:
CNN.fit(np.array(x_train).astype(float), np.array(y_train), batch_size=1, epochs=10)
CNN.evaluate(np.array(x_test).astype(float), np.array(y_test))

10/10 [==============================] - 8s 759ms/step - loss: 1.4060 - accuracy: 0.4566


[1.4059847593307495, 0.4565916359424591]

In [38]:
y_predict = [np.argmax(CNN.predict(np.array([cv2.resize(img, (160, 120), interpolation=cv2.INTER_AREA)]).astype(float))) for img in x_test]
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.46      1.00      0.63       140
           1       0.00      0.00      0.00        59
           2       0.00      0.00      0.00        37
           3       0.00      0.00      0.00        34
           4       1.00      0.05      0.09        41

    accuracy                           0.46       311
   macro avg       0.29      0.21      0.14       311
weighted avg       0.34      0.46      0.29       311



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
method_CNN = lambda img: np.argmax(CNN.predict(np.array([cv2.resize(img, (160, 120), interpolation=cv2.INTER_AREA)]).astype(float)))

---
### online predict

In [ ]:
def onlinePredict(method):
    # empty: 0, up: 1, down: 2, left: 3, right: 4
    action = {0: 'empty', 1: 'up', 2: 'down', 3: 'right', 4: 'left'}
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot open camera")
        exit()
        
    while(True):
        # 擷取影像
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        img = frame[:, ::-1]
        # 顯示圖片並預測
        cv2.imshow('live', img)
        predict = action[method(frame)]
        print(predict)
        
        # 按下 q 鍵離開迴圈
        if cv2.waitKey(1) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    return

In [ ]:
methods = {'SVM': method_SVM, 'CNN': method_CNN, 'Mediapipe': method_mediapipe}
onlinePredict(methods['SVM'])